In [1]:
!pip install --break-system-packages git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-m7sxpqs6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-m7sxpqs6
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=d4d0737090ffea034bb76c52c87dafa03268c99c754e101840eb59e8a6ea72b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-j6rg405s/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper


In [2]:
!pip install --break-system-packages pyannote.audio torchaudio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.2/825.2 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# # app_gradio.py
# # ---------------------------------------------------------
# # Whisper + (optional) Pyannote UI with "Dialogue" and "Report Analysis"
# # ---------------------------------------------------------
# # Run:
# #   python app_gradio.py
# # In Colab:
# #   from app_gradio import demo; demo.launch(share=True)

# import os
# import re
# import json
# import subprocess
# from datetime import datetime
# from functools import lru_cache
# from typing import List, Dict, Any

# import gradio as gr
# import torch
# import pandas as pd
# from openpyxl import Workbook
# from openpyxl.styles import Font, Alignment, Border, Side
# from openpyxl.utils import get_column_letter
# from html import escape

# # ML libs
# import whisper
# from pyannote.audio import Pipeline

# # -----------------------
# # Config
# # -----------------------
# DEFAULT_MODEL_NAME = os.getenv("WHISPER_MODEL", "large-v3")
# SAMPLE_RATE_HZ = 16000

# # Default HF token (falls back to your hardcoded value if env var is empty)
# DEFAULT_HF_TOKEN = os.getenv(
#     "HUGGING_FACE_TOKEN",
#     "hf_"
# )

# # Known agents list (added from recent updates)
# KNOWN_AGENTS = [
#     "Jaya Parkash", "Chandru", "Sneha", "Kowsalya", "Swathi",
#     "Arogya Marry", "Delphina", "Aesu Marry"
# ]

# # -----------------------
# # FFmpeg / audio utils
# # -----------------------
# def get_audio_duration(audio_path: str) -> float:
#     try:
#         cmd = ["ffprobe", "-v", "error", "-show_entries", "format=duration",
#                "-of", "default=noprint_wrappers=1:nokey=1", audio_path]
#         out = subprocess.run(cmd, capture_output=True, text=True, check=True)
#         return float(out.stdout.strip())
#     except Exception:
#         return 0.0

# def _ffmpeg_run(cmd: List[str]) -> bool:
#     try:
#         subprocess.run(cmd, check=True, capture_output=True, text=True)
#         return True
#     except subprocess.CalledProcessError:
#         return False

# def audio_preprocessing_v1(input_path, output_path):
#     cmd = [
#         "ffmpeg", "-i", input_path,
#         "-acodec", "pcm_s16le", "-ac", "1", "-ar", str(SAMPLE_RATE_HZ),
#         "-af", "loudnorm=I=-23:TP=-2:LRA=7,highpass=f=80,lowpass=f=8000,afftdn=nr=10",
#         "-y", output_path
#     ]
#     return _ffmpeg_run(cmd)

# def audio_preprocessing_v2(input_path, output_path):
#     cmd = [
#         "ffmpeg", "-i", input_path,
#         "-acodec", "pcm_s16le", "-ac", "1", "-ar", str(SAMPLE_RATE_HZ),
#         "-af", "loudnorm=I=-23:TP=-2,highpass=f=100",
#         "-y", output_path
#     ]
#     return _ffmpeg_run(cmd)

# def audio_preprocessing_v3(input_path, output_path):
#     cmd = [
#         "ffmpeg", "-i", input_path,
#         "-acodec", "pcm_s16le", "-ac", "1", "-ar", str(SAMPLE_RATE_HZ),
#         "-af", "loudnorm",
#         "-y", output_path
#     ]
#     return _ffmpeg_run(cmd)

# def audio_preprocessing_v4(input_path, output_path):
#     cmd = [
#         "ffmpeg", "-i", input_path,
#         "-acodec", "pcm_s16le", "-ac", "1", "-ar", str(SAMPLE_RATE_HZ),
#         "-y", output_path
#     ]
#     return _ffmpeg_run(cmd)

# def smart_audio_preprocessing(input_path, output_path, log: List[str]) -> bool:
#     original_dur = get_audio_duration(input_path)
#     log.append(f"Original audio duration: {original_dur:.2f}s")

#     methods = [
#         ("Advanced (afftdn)", audio_preprocessing_v1),
#         ("Simplified", audio_preprocessing_v2),
#         ("Basic", audio_preprocessing_v3),
#         ("Minimal", audio_preprocessing_v4),
#     ]
#     for idx, (name, fn) in enumerate(methods, 1):
#         log.append(f"→ Trying {name} preprocessing...")
#         if fn(input_path, output_path) and os.path.exists(output_path):
#             processed_dur = get_audio_duration(output_path)
#             log.append(f"Processed duration: {processed_dur:.2f}s")
#             if abs(original_dur - processed_dur) < 1.0:
#                 log.append(f"✅ Preprocessing successful with method {idx}: {name}")
#                 return True
#             else:
#                 log.append(f"⚠️ Duration mismatch on method {idx}; trying next...")
#     log.append("❌ All preprocessing methods failed.")
#     return False

# # -----------------------
# # Text cleanup / repetition control
# # -----------------------
# def post_process_text(text: str) -> str:
#     if not text:
#         return ""
#     words = text.split()
#     cleaned_words = []
#     i = 0
#     while i < len(words):
#         current = words[i].lower()
#         count = 1
#         j = i + 1
#         while j < len(words) and words[j].lower() == current:
#             count += 1
#             j += 1
#         keep = min(count, 2) if count <= 3 else 1
#         for _ in range(keep):
#             cleaned_words.append(words[i])
#         i += count
#     text = " ".join(cleaned_words)

#     filler_sounds = ["uh", "um", "mm", "hmm", "ah", "oh", "huh", "ha ha"]
#     soft_fillers = ["okay okay", "yes yes", "yes yes yes", "i mean", "you know", "like like", "ok ok"]
#     for f in filler_sounds + soft_fillers:
#         text = re.sub(rf'\b{re.escape(f)}\b', '', text, flags=re.IGNORECASE)

#     corrections = {
#         'access max life': 'Axis Max Life',
#         'axis max life': 'Axis Max Life',
#         'g pay': 'GPay',
#         'google pay': 'Google Pay',
#         'phone pay': 'PhonePe',
#         'phone pe': 'PhonePe',
#         'pay tm': 'Paytm',
#         'net banking': 'netbanking',
#         'some assured': 'sum assured',
#         'premium do': 'premium due',
#         'do date': 'due date',
#         'okay sir': 'Okay sir',
#     }
#     text_lower = text.lower()
#     for wrong, correct in corrections.items():
#         text_lower = text_lower.replace(wrong, correct)

#     text_lower = re.sub(r'\brs[.]?\s*', '₹', text_lower)
#     text_lower = re.sub(r'\s{2,}', ' ', text_lower)
#     text_lower = re.sub(r'[,]{2,}', ',', text_lower)
#     text_lower = re.sub(r'\s+,', ',', text_lower)
#     text_lower = re.sub(r'\s+\.', '.', text_lower)
#     text_lower = re.sub(r'\s+[!?]', lambda m: m.group(0).strip(), text_lower)
#     text_lower = re.sub(r'(^|[.!?]\s+)([a-z])', lambda m: m.group(1) + m.group(2).upper(), text_lower)
#     return text_lower.strip()

# def detect_and_remove_repetitions(segments, max_repetition_ratio=0.3, log=None):
#     if log is None:
#         log = []
#     log.append("🔍 Aggressive repetition detection started...")
#     cleaned = []
#     for seg in segments:
#         text = seg["text"].strip()
#         words = text.split()
#         if len(words) < 2:
#             continue

#         wc = {}
#         for w in words:
#             wl = w.lower().strip('.,!?')
#             wc[wl] = wc.get(wl, 0) + 1
#         max_count = max(wc.values()) if wc else 0
#         dominance = max_count / len(words) if words else 0
#         if dominance > 0.4:
#             log.append(f"🚫 Drop (dominant word): {text[:50]}...")
#             continue

#         consec = 0
#         max_consec = 0
#         for j in range(1, len(words)):
#             if words[j].lower().strip('.,!?') == words[j-1].lower().strip('.,!?'):
#                 consec += 1
#                 max_consec = max(max_consec, consec + 1)
#             else:
#                 consec = 0
#         if max_consec > 3:
#             log.append(f"🚫 Drop (consecutive repeats): {text[:50]}...")
#             continue

#         repetitive = False
#         for phrase_len in range(2, min(len(words)//3 + 1, 8)):
#             for start in range(len(words) - phrase_len * 2 + 1):
#                 p1 = ' '.join(words[start:start+phrase_len]).lower()
#                 p2 = ' '.join(words[start+phrase_len:start+phrase_len*2]).lower()
#                 if p1 == p2:
#                     coverage = (phrase_len * 2) / len(words)
#                     if coverage > max_repetition_ratio:
#                         log.append(f"🚫 Drop (pattern repeat): {text[:50]}...")
#                         repetitive = True
#                         break
#             if repetitive:
#                 break
#         if repetitive:
#             continue

#         is_dup = False
#         for prev in cleaned[-5:]:
#             prev_words = prev['text'].lower().split()
#             curr_words = [w.lower() for w in words]
#             if prev_words and curr_words:
#                 prev_set = set(prev_words)
#                 curr_set = set(curr_words)
#                 inter = len(prev_set & curr_set)
#                 union = len(prev_set | curr_set)
#                 sim = inter / union if union else 0
#                 if sim > 0.7 and abs(len(prev_words) - len(curr_words)) < 5:
#                     log.append(f"🚫 Drop (near-duplicate): {text[:50]}...")
#                     is_dup = True
#                     break
#         if is_dup:
#             continue

#         cleaned.append(seg)
#     log.append(f"📊 Cleaned {len(segments)} → {len(cleaned)} segments")
#     return cleaned

# # -----------------------
# # Whisper + diarization
# # -----------------------
# @lru_cache(maxsize=1)
# def get_whisper_model(name: str):
#     return whisper.load_model(name)

# def enhanced_whisper_transcription(audio_path: str, model_name: str, log) -> Dict[str, Any]:
#     log.append("--- Enhanced Whisper Transcription ---")
#     # prompt = (
#     #     "This is a customer support call for Axis Maxlife Insurance. "
#     #     "We will discuss policy numbers, due date, fund value, sum assured, late fee, "
#     #     "and payment methods such as Google Pay, PhonePe, Paytm and netbanking."
#     # )
#     agents_str = ", ".join(KNOWN_AGENTS)
#     prompt = (
#         f"This is a customer support call from Axis Maxlife Insurance in Tamil regarding insurance renewal call. "
#         f"Agents are always one of these: {agents_str}. Use exact names like 'Jaya Parkash' or 'Swathi' "
#         f"when they introduce themselves. Discuss policy numbers, due dates, fund values, sum assured, late fees, "
#         f"and payment methods such as Google Pay, PhonePe, Paytm, netbanking."
#     )
#     model = get_whisper_model(model_name)
#     result = model.transcribe(
#         audio_path,
#         language="ta",
#         task="translate",
#         temperature=0.0,
#         beam_size=5,
#         patience=1.2,
#         condition_on_previous_text=False,
#         no_speech_threshold=0.8,
#         compression_ratio_threshold=2.0,
#         logprob_threshold=-0.35,
#         word_timestamps=False,
#         initial_prompt=prompt,
#         verbose=True,
#     )
#     log.append("✅ Whisper transcription done.")
#     return result

# def run_diarization(wav_path: str, hf_token: str, log):
#     try:
#         token = hf_token or os.getenv("HUGGING_FACE_TOKEN", "")
#         if not token:
#             log.append("ℹ️ No HF token provided: skipping diarization (Speaker_Unknown).")
#             return None
#         # Support both old/new arg names
#         try:
#             pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=token)
#         except TypeError:
#             pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", token=token)
#         if torch.cuda.is_available():
#             pipeline.to(torch.device("cuda"))
#             log.append("✅ Using GPU for diarization.")
#         diarization = pipeline(wav_path)
#         log.append("✅ Speaker diarization completed.")
#         return diarization
#     except Exception as e:
#         log.append(f"⚠️ Diarization failed: {e}")
#         return None

# # -----------------------
# # Dialogue building/formatting
# # -----------------------
# def get_dominant_speaker(start_time, end_time, diarization_result):
#     if not diarization_result:
#         return "Speaker_Unknown"
#     speakers = {}
#     for segment, _, speaker in diarization_result.itertracks(yield_label=True):
#         overlap_start = max(start_time, segment.start)
#         overlap_end = min(end_time, segment.end)
#         dur = max(0.0, overlap_end - overlap_start)
#         if dur > 0:
#             speakers[speaker] = speakers.get(speaker, 0.0) + dur
#     return max(speakers, key=speakers.get) if speakers else "Speaker_Unknown"

# def build_dialogue(processed_segments, diarization):
#     dialogue = []
#     current_speaker = None
#     current_texts = []
#     current_start = 0.0
#     current_end = 0.0
#     for seg in processed_segments:
#         start = seg['start']
#         end = seg['end']
#         text = seg['text'].strip()
#         speaker = get_dominant_speaker(start, end, diarization)
#         if (speaker == current_speaker and current_speaker and (start - current_end) < 3.0):
#             current_texts.append(text)
#             current_end = end
#         else:
#             if current_speaker and current_texts:
#                 combined = ' '.join(current_texts)
#                 if len(combined.strip()) > 10:
#                     dialogue.append({
#                         'speaker': current_speaker,
#                         'text': combined,
#                         'start_time': current_start,
#                         'end_time': current_end
#                     })
#             current_speaker = speaker
#             current_texts = [text]
#             current_start = start
#             current_end = end
#     if current_speaker and current_texts:
#         combined = ' '.join(current_texts)
#         if len(combined.strip()) > 10:
#             dialogue.append({
#                 'speaker': current_speaker,
#                 'text': combined,
#                 'start_time': current_start,
#                 'end_time': current_end
#             })
#     return dialogue

# def format_dialogue_markdown(dialogue: List[Dict[str, Any]]) -> str:
#     if not dialogue:
#         return "_No substantial dialogue detected._"
#     lines = ["## 🎭 Dialogue", ""]
#     for d in dialogue:
#         ts = f"[{d['start_time']:.1f}s - {d['end_time']:.1f}s]"
#         lines.append(f"**{d['speaker']} {ts}:**")
#         lines.append(f"{d['text']}")
#         lines.append("")
#     return "\n".join(lines)

# # -----------------------
# # Field extraction from Dialogue
# # -----------------------
# def _norm(text: str) -> str:
#     return re.sub(r"\s+", " ", text).strip()

# def extract_fields_from_dialogue(dialogue):
#     full = " ".join(_norm(d["text"]) for d in dialogue).lower()
#     get = {}

#     m = re.search(r"(policy\s*(no|number)\s*[:\-]?\s*)(\d{6,12})", full)
#     get["Policy No"] = m.group(3) if m else "—"

#     m = re.search(r"(this is|i am|speaking)\s+([a-zA-Z][a-zA-Z.\s]{2,30})\s*(from|with|here)", full)
#     get["Associate Name"] = m.group(2).title().strip() if m else "—"

#     m = re.search(r"(mr|mrs|ms)\.?\s+([a-zA-Z][a-zA-Z.\s]{2,30})", full)
#     get["Customer Name"] = m.group(2).title().strip() if m else "—"

#     m = re.search(r"(?:mobile|phone|contact)\s*(no|number)?\s*[:\-]?\s*(\+?91[\s\-]?)?(\d{10})", full)
#     get["Mobile Number"] = (("+" + "91 ") if m and m.group(2) else "") + (m.group(3) if m else "—")

#     m = re.search(r"(due\s*date|premium\s*due)\s*[:\-]?\s*([0-3]?\d[\/\-][01]?\d(?:[\/\-]\d{2,4})?|[0-3]?\d\s+[a-zA-Z]{3,9}\s+\d{4})", full)
#     get["Due Date"] = m.group(2) if m else "—"

#     m = re.search(r"(?:premium\s*(amount|due)?\s*[:\-]?\s*)(?:₹|rs\.?\s*)\s*([0-9,]+(\.\d{1,2})?)", full)
#     get["Premium Amount"] = "₹" + m.group(2) if m else "—"

#     m = re.search(r"(sum\s*assured)\s*[:\-]?\s*(?:₹|rs\.?\s*)?\s*([0-9,]+)", full)
#     get["Sum Assured"] = "₹" + m.group(2) if m else "—"

#     m = re.search(r"(fund\s*value)\s*[:\-]?\s*(?:₹|rs\.?\s*)?\s*([0-9,]+(\.\d{1,2})?)", full)
#     get["Fund Value"] = "₹" + m.group(2) if m else "—"

#     m = re.search(r"(policy\s*status)\s*[:\-]?\s*(active|inactive|lapsed|in force|paid up)", full)
#     get["Policy Status"] = m.group(2).title() if m else "—"

#     m = re.search(r"(payment\s*(mode|method))\s*[:\-]?\s*(cash|card|netbanking|upi|google\s*pay|gpay|phonepe|paytm)", full)
#     get["Payment Mode"] = (m.group(3).title().replace("Gpay","GPay").replace("Google Pay","Google Pay").replace("Phonepe","PhonePe")) if m else "—"

#     return get

# # -----------------------
# # Report UI
# # -----------------------
# def build_report_html(fields: dict, title: str = "BRU - CQ Sheet") -> str:
#     """Render a simple HTML report (title + label/value rows) like your reference sheet."""
#     style = """
#     <style>
#       .vs-report{max-width:860px;margin:10px auto 24px;background:#111319;border:1px solid #2a2f3a;
#                  border-radius:14px;padding:18px 22px;font-family:ui-sans-serif,system-ui,Segoe UI,Roboto,Helvetica,Arial;
#                  color:#e7e9ee}
#       .vs-title{font-size:20px;font-weight:700;margin:4px 0 14px;color:#f2f4f8}
#       .vs-grid{width:100%;border-collapse:separate;border-spacing:0 8px}
#       .vs-grid th{width:30%;text-align:left;color:#aeb6c2;font-weight:600;padding:10px 12px;background:#0c0e13;border-radius:10px 0 0 10px;border:1px solid #272c36;border-right:none}
#       .vs-grid td{padding:10px 12px;background:#0c0f16;border-radius:0 10px 10px 0;border:1px solid #272c36;border-left:none}
#       .vs-section{margin-top:18px}
#       .vs-sec-h{font-size:14px;font-weight:700;margin:8px 0 6px;color:#d9dde6}
#       .vs-muted{color:#9aa3af}
#       .vs-foot{margin-top:12px;color:#9aa3af;font-size:12px}
#     </style>
#     """
#     rows = "".join(
#         f"<tr><th>{escape(k)}</th><td>{escape(str(v) if v is not None else '—')}</td></tr>"
#         for k, v in fields.items()
#     )
#     html = f"""
#     {style}
#     <div class="vs-report">
#       <div class="vs-title">{escape(title)}</div>
#       <table class="vs-grid">{rows}</table>
#       <div class="vs-section">
#         <div class="vs-sec-h">Call Highlights</div>
#         <div class="vs-muted">We’ll auto-fill specific highlights later (issues, commitments, amounts, dates).</div>
#       </div>
#       <div class="vs-section">
#         <div class="vs-sec-h">Compliance / Quality</div>
#         <div class="vs-muted">Checklist to be added per your requirements.</div>
#       </div>
#       <div class="vs-foot">Generated live from the audio-derived dialogue.</div>
#     </div>
#     """
#     return html

# def generate_report_html_from_state(dialogue, fields):
#     """Return HTML (no file)."""
#     if not dialogue:
#         return "<div class='vs-report'><div class='vs-title'>BRU - CQ Sheet</div><div class='vs-muted'>⚠️ Process an audio first to build the report.</div></div>"
#     if not fields or not isinstance(fields, dict):
#         fields = extract_fields_from_dialogue(dialogue)
#     return build_report_html(fields, title="BRU - CQ Sheet")

# # -----------------------
# # Gradio app
# # -----------------------
# def transcribe_and_diarize(audio_file, model_name, enable_diarization, hf_token):
#     logs = []
#     if audio_file is None:
#         return "_Please upload an audio file._", "{}", "No audio provided.", [], {}

#     input_path = audio_file
#     cleaned_path = os.path.join(os.path.dirname(input_path), "cleaned_audio_for_asr_and_diarization.wav")

#     ok = smart_audio_preprocessing(input_path, cleaned_path, logs)
#     if not ok:
#         return "_Preprocessing failed._", "{}", "\n".join(logs), [], {}

#     try:
#         whisper_result = enhanced_whisper_transcription(cleaned_path, model_name, logs)
#     except Exception as e:
#         logs.append(f"❌ Whisper transcription failed: {e}")
#         return "_Transcription failed._", "{}", "\n".join(logs), [], {}

#     segments = (whisper_result.get("segments") or [])
#     cleaned_segments = detect_and_remove_repetitions(segments, log=logs)

#     processed_segments = []
#     for seg in cleaned_segments:
#         processed_text = post_process_text(seg['text'])
#         if processed_text.strip() and len(processed_text.strip()) > 5:
#             seg_copy = dict(seg)
#             seg_copy['text'] = processed_text
#             processed_segments.append(seg_copy)

#     diarization = None
#     if enable_diarization:
#         diarization = run_diarization(cleaned_path, hf_token, logs)

#     dialogue = build_dialogue(processed_segments, diarization)
#     fields = extract_fields_from_dialogue(dialogue)

#     md = format_dialogue_markdown(dialogue)
#     output_payload = {
#         "metadata": {
#             "total_duration": whisper_result.get('duration', 0),
#             "total_speakers": len(set(d['speaker'] for d in dialogue)),
#             "total_segments": len(dialogue),
#             "model_used": f"whisper-{model_name}",
#             "processing_successful": True,
#             "anti_repetition_applied": True,
#             "diarization_enabled": bool(enable_diarization),
#         },
#         "dialogue": dialogue,
#         "fields": fields,
#     }
#     return md, json.dumps(output_payload, indent=2, ensure_ascii=False), "\n".join(logs), dialogue, fields

# def _enable_build_button(dialogue):
#     # enable only if we actually have a dialogue from Process
#     return gr.update(interactive=bool(dialogue))

# # Build UI
# latest_dialogue_state = gr.State([])
# latest_fields_state = gr.State({})

# with gr.Blocks(title="Speech Analysis - QA") as demo:
#     gr.Markdown("# 🗣️ Whisper Call Transcriber → 📑 Report\nUpload a call recording, get a clean dialogue and a report like your reference sheet.")
#     with gr.Row():
#         audio = gr.Audio(type="filepath", label="Upload audio (wav/mp3/m4a...)")
#     with gr.Row():
#         model_name = gr.Dropdown(choices=["large-v3","large", "medium", "small", "base", "tiny"], value=DEFAULT_MODEL_NAME, label="Whisper model")
#         enable_diar = gr.Checkbox(value=True, label="Enable diarization (Pyannote)")
#         hf_token = gr.Textbox(value=DEFAULT_HF_TOKEN, label="Hugging Face token (required if diarization enabled)", type="password", placeholder="hf_xxx...")
#     run_btn = gr.Button("▶️ Process")

#     with gr.Tab("Dialogue"):
#         dialogue_md = gr.Markdown()

#     with gr.Tab("Report Analysis"):
#         gr.Markdown("Build a report **from the audio-derived dialogue** and render it here.")
#         build_btn = gr.Button("📄 Generate Report from Dialogue")  # <- remove interactive=False
#         report_html = gr.HTML(value="<div class='vs-muted'>_No report yet._</div>")

#     with gr.Tab("JSON"):
#         json_out = gr.Code(language="json", label="Output JSON")

#     with gr.Tab("Logs"):
#         logs_out = gr.Textbox(label="Processing Logs", lines=18)
#         dialogue_inline = gr.Markdown(value="_Dialogue will appear here_")

#     # Wire processing
#     run_btn.click(
#         transcribe_and_diarize,
#         inputs=[audio, model_name, enable_diar, hf_token],
#         outputs=[dialogue_md, json_out, logs_out, latest_dialogue_state, latest_fields_state]
#     ).then(
#         # optional: keep or remove this; button is always clickable now
#         _enable_build_button, inputs=[latest_dialogue_state], outputs=[build_btn]
#     ).then(
#         lambda md, *_: md,  # mirror Dialogue under Logs
#         inputs=[dialogue_md],
#         outputs=[dialogue_inline]
#     ).then(
#         # NEW: generate the report HTML immediately after processing
#         generate_report_html_from_state,
#         inputs=[latest_dialogue_state, latest_fields_state],
#         outputs=[report_html]
#     )

#     # Wire report builder (render HTML, no download)
#     build_btn.click(
#         generate_report_html_from_state,
#         inputs=[latest_dialogue_state, latest_fields_state],
#         outputs=[report_html]
#     )


# if __name__ == "__main__":
#     # In notebooks/Colab, letting Gradio choose a free port is easiest
#     gr.close_all()
#     demo.launch(share=True)


In [ ]:
# # Show what's on 7860
# !lsof -i:7860 || true
# # Kill anything still bound there
# !fuser -k 7860/tcp || true

# import gradio as gr
# gr.close_all()
# demo.launch(share=True)  # or pick another port like 7861


In [ ]:
#sample report structure

In [ ]:
# # app_gradio.py
# # ---------------------------------------------------------
# # Whisper + (optional) Pyannote UI with "Dialogue" and "Report Analysis"
# # ---------------------------------------------------------

# import os, re, json, subprocess
# from functools import lru_cache
# from typing import List, Dict, Any
# from html import escape

# import gradio as gr
# import torch
# import whisper
# from pyannote.audio import Pipeline

# # -----------------------
# # Config
# # -----------------------
# DEFAULT_MODEL_NAME = os.getenv("WHISPER_MODEL", "large-v3")
# SAMPLE_RATE_HZ = 16000
# DEFAULT_HF_TOKEN = os.getenv("HUGGING_FACE_TOKEN", "hf_")

# KNOWN_AGENTS = [
#     "Jaya Parkash", "Chandru", "Sneha", "Kowsalya", "Swathi",
#     "Arogya Marry", "Delphina", "Aesu Marry"
# ]

# # -----------------------
# # FFmpeg / audio utils
# # -----------------------
# def get_audio_duration(audio_path: str) -> float:
#     try:
#         out = subprocess.run(
#             ["ffprobe","-v","error","-show_entries","format=duration","-of","default=noprint_wrappers=1:nokey=1",audio_path],
#             capture_output=True, text=True, check=True
#         )
#         return float(out.stdout.strip())
#     except Exception:
#         return 0.0

# def _ffmpeg_run(cmd: List[str]) -> bool:
#     try:
#         subprocess.run(cmd, check=True, capture_output=True, text=True)
#         return True
#     except subprocess.CalledProcessError:
#         return False

# def audio_preprocessing_v1(i,o):
#     return _ffmpeg_run(["ffmpeg","-i",i,"-acodec","pcm_s16le","-ac","1","-ar",str(SAMPLE_RATE_HZ),
#                         "-af","loudnorm=I=-23:TP=-2:LRA=7,highpass=f=80,lowpass=f=8000,afftdn=nr=10","-y",o])
# def audio_preprocessing_v2(i,o):
#     return _ffmpeg_run(["ffmpeg","-i",i,"-acodec","pcm_s16le","-ac","1","-ar",str(SAMPLE_RATE_HZ),
#                         "-af","loudnorm=I=-23:TP=-2,highpass=f=100","-y",o])
# def audio_preprocessing_v3(i,o):
#     return _ffmpeg_run(["ffmpeg","-i",i,"-acodec","pcm_s16le","-ac","1","-ar",str(SAMPLE_RATE_HZ),
#                         "-af","loudnorm","-y",o])
# def audio_preprocessing_v4(i,o):
#     return _ffmpeg_run(["ffmpeg","-i",i,"-acodec","pcm_s16le","-ac","1","-ar",str(SAMPLE_RATE_HZ),"-y",o])

# def smart_audio_preprocessing(input_path, output_path, log: List[str]) -> bool:
#     original_dur = get_audio_duration(input_path)
#     log.append(f"Original audio duration: {original_dur:.2f}s")
#     methods = [("Advanced (afftdn)",audio_preprocessing_v1),("Simplified",audio_preprocessing_v2),
#                ("Basic",audio_preprocessing_v3),("Minimal",audio_preprocessing_v4)]
#     for idx,(name,fn) in enumerate(methods,1):
#         log.append(f"→ Trying {name} preprocessing...")
#         if fn(input_path, output_path) and os.path.exists(output_path):
#             processed_dur = get_audio_duration(output_path)
#             log.append(f"Processed duration: {processed_dur:.2f}s")
#             if abs(original_dur - processed_dur) < 1.0:
#                 log.append(f"✅ Preprocessing successful with method {idx}: {name}")
#                 return True
#             else:
#                 log.append(f"⚠️ Duration mismatch on method {idx}; trying next...")
#     log.append("❌ All preprocessing methods failed.")
#     return False

# # -----------------------
# # Text cleanup / repetition control
# # -----------------------
# def post_process_text(text: str) -> str:
#     if not text: return ""
#     words = text.split()
#     cleaned_words, i = [], 0
#     while i < len(words):
#         cur = words[i].lower(); count = 1; j = i+1
#         while j < len(words) and words[j].lower() == cur: count += 1; j += 1
#         cleaned_words.extend([words[i]] * (min(count,2) if count <= 3 else 1)); i += count
#     text = " ".join(cleaned_words)

#     filler = ["uh","um","mm","hmm","ah","oh","huh","ha ha",
#               "okay okay","yes yes","yes yes yes","i mean","you know","like like","ok ok"]
#     for f in filler:
#         text = re.sub(rf'\b{re.escape(f)}\b','',text,flags=re.IGNORECASE)

#     repl = {'access max life':'Axis Max Life','axis max life':'Axis Max Life','g pay':'GPay',
#             'google pay':'Google Pay','phone pay':'PhonePe','phone pe':'PhonePe','pay tm':'Paytm',
#             'net banking':'netbanking','some assured':'sum assured','premium do':'premium due',
#             'do date':'due date','okay sir':'Okay sir'}
#     low = text.lower()
#     for w,c in repl.items(): low = low.replace(w,c)
#     low = re.sub(r'\brs[.]?\s*','₹',low)
#     low = re.sub(r'\s{2,}',' ',low)
#     low = re.sub(r'[,]{2,}',',',low)
#     low = re.sub(r'\s+,',',',low)
#     low = re.sub(r'\s+\.','.',low)
#     low = re.sub(r'\s+[!?]', lambda m:m.group(0).strip(), low)
#     low = re.sub(r'(^|[.!?]\s+)([a-z])', lambda m:m.group(1)+m.group(2).upper(), low)
#     return low.strip()

# def detect_and_remove_repetitions(segments, max_repetition_ratio=0.3, log=None):
#     log = log or []; log.append("🔍 Aggressive repetition detection started...")
#     cleaned = []
#     for seg in segments:
#         text = seg["text"].strip(); words = text.split()
#         if len(words) < 2: continue
#         wc = {}
#         for w in words:
#             wl = w.lower().strip('.,!?'); wc[wl] = wc.get(wl,0)+1
#         dominance = (max(wc.values())/len(words)) if wc else 0
#         if dominance > 0.4: log.append(f"🚫 Drop (dominant word): {text[:50]}..."); continue

#         consec=max_consec=0
#         for j in range(1,len(words)):
#             if words[j].lower().strip('.,!?') == words[j-1].lower().strip('.,!?'):
#                 consec += 1; max_consec = max(max_consec, consec+1)
#             else: consec = 0
#         if max_consec > 3: log.append(f"🚫 Drop (consecutive repeats): {text[:50]}..."); continue

#         repetitive=False
#         for L in range(2, min(len(words)//3+1,8)):
#             for s in range(len(words)-L*2+1):
#                 if ' '.join(words[s:s+L]).lower() == ' '.join(words[s+L:s+2*L]).lower():
#                     if (L*2)/len(words) > max_repetition_ratio:
#                         log.append(f"🚫 Drop (pattern repeat): {text[:50]}..."); repetitive=True; break
#             if repetitive: break
#         if repetitive: continue

#         is_dup=False
#         for prev in cleaned[-5:]:
#             a,b=set(prev['text'].lower().split()), set(w.lower() for w in words)
#             if a|b and (len(a&b)/len(a|b))>0.7 and abs(len(a)-len(b))<5:
#                 log.append(f"🚫 Drop (near-duplicate): {text[:50]}..."); is_dup=True; break
#         if is_dup: continue
#         cleaned.append(seg)
#     log.append(f"📊 Cleaned {len(segments)} → {len(cleaned)} segments")
#     return cleaned

# # -----------------------
# # Whisper + diarization
# # -----------------------
# @lru_cache(maxsize=1)
# def get_whisper_model(name: str):
#     return whisper.load_model(name)

# def enhanced_whisper_transcription(audio_path: str, model_name: str, log) -> Dict[str, Any]:
#     log.append("--- Enhanced Whisper Transcription ---")
#     agents_str = ", ".join(KNOWN_AGENTS)
#     prompt = (
#         f"This is a customer support call from Axis Maxlife Insurance in Tamil regarding insurance renewal call. "
#         f"Agents are always one of these: {agents_str}. Use exact names like 'Jaya Parkash' or 'Swathi'. "
#         f"Discuss policy numbers, due dates, fund values, sum assured, late fees, and payment methods such as "
#         f"Google Pay, PhonePe, Paytm, netbanking."
#     )
#     model = get_whisper_model(model_name)
#     result = model.transcribe(
#         audio_path, language="ta", task="translate", temperature=0.0,
#         beam_size=5, patience=1.2, condition_on_previous_text=False,
#         no_speech_threshold=0.8, compression_ratio_threshold=2.0,
#         logprob_threshold=-0.35, word_timestamps=False,
#         initial_prompt=prompt, verbose=True,
#     )
#     log.append("✅ Whisper transcription done.")
#     return result

# def run_diarization(wav_path: str, hf_token: str, log):
#     try:
#         token = hf_token or os.getenv("HUGGING_FACE_TOKEN", "")
#         if not token:
#             log.append("ℹ️ No HF token provided: skipping diarization (Speaker_Unknown).")
#             return None
#         try:
#             pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=token)
#         except TypeError:
#             pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", token=token)
#         if torch.cuda.is_available():
#             pipeline.to(torch.device("cuda")); log.append("✅ Using GPU for diarization.")
#         diarization = pipeline(wav_path); log.append("✅ Speaker diarization completed.")
#         return diarization
#     except Exception as e:
#         log.append(f"⚠️ Diarization failed: {e}"); return None

# # -----------------------
# # Dialogue + fields
# # -----------------------
# def get_dominant_speaker(start_time, end_time, diarization_result):
#     if not diarization_result: return "Speaker_Unknown"
#     speakers = {}
#     for segment, _, speaker in diarization_result.itertracks(yield_label=True):
#         overlap_start = max(start_time, segment.start)
#         overlap_end = min(end_time, segment.end)
#         dur = max(0.0, overlap_end - overlap_start)
#         if dur > 0: speakers[speaker] = speakers.get(speaker, 0.0) + dur
#     return max(speakers, key=speakers.get) if speakers else "Speaker_Unknown"

# def build_dialogue(processed_segments, diarization):
#     out=[]; cur_spk=None; cur_texts=[]; cur_start=0.0; cur_end=0.0
#     for seg in processed_segments:
#         start,end,txt = seg['start'], seg['end'], seg['text'].strip()
#         spk = get_dominant_speaker(start,end,diarization)
#         if (spk==cur_spk and cur_spk and (start-cur_end)<3.0):
#             cur_texts.append(txt); cur_end=end
#         else:
#             if cur_spk and cur_texts:
#                 c=' '.join(cur_texts)
#                 if len(c.strip())>10: out.append({'speaker':cur_spk,'text':c,'start_time':cur_start,'end_time':cur_end})
#             cur_spk,cur_texts,cur_start,cur_end = spk,[txt],start,end
#     if cur_spk and cur_texts:
#         c=' '.join(cur_texts)
#         if len(c.strip())>10: out.append({'speaker':cur_spk,'text':c,'start_time':cur_start,'end_time':cur_end})
#     return out

# def format_dialogue_markdown(dialogue: List[Dict[str, Any]]) -> str:
#     if not dialogue: return "_No substantial dialogue detected._"
#     lines=["## 🎭 Dialogue",""]
#     for d in dialogue:
#         lines.append(f"**{d['speaker']} [{d['start_time']:.1f}s - {d['end_time']:.1f}s]:**")
#         lines.append(d['text']); lines.append("")
#     return "\n".join(lines)

# def _norm(t: str) -> str: return re.sub(r"\s+"," ",t).strip()

# def extract_fields_from_dialogue(dialogue):
#     full = " ".join(_norm(d["text"]) for d in dialogue).lower()
#     g={}
#     m=re.search(r"(policy\s*(no|number)\s*[:\-]?\s*)(\d{6,12})",full); g["Policy No"]=m.group(3) if m else "—"
#     m=re.search(r"(this is|i am|speaking)\s+([a-zA-Z][a-zA-Z.\s]{2,30})\s*(from|with|here)",full); g["Associate Name"]=m.group(2).title().strip() if m else "—"
#     m=re.search(r"(mr|mrs|ms)\.?\s+([a-zA-Z][a-zA-Z.\s]{2,30})",full); g["Customer Name"]=m.group(2).title().strip() if m else "—"
#     m=re.search(r"(?:mobile|phone|contact)\s*(no|number)?\s*[:\-]?\s*(\+?91[\s\-]?)?(\d{10})",full); g["Mobile Number"]=(("+"+"91 ") if m and m.group(2) else "")+(m.group(3) if m else "—")
#     m=re.search(r"(due\s*date|premium\s*due)\s*[:\-]?\s*([0-3]?\d[\/\-][01]?\d(?:[\/\-]\d{2,4})?|[0-3]?\d\s+[a-zA-Z]{3,9}\s+\d{4})",full); g["Due Date"]=m.group(2) if m else "—"
#     m=re.search(r"(?:premium\s*(amount|due)?\s*[:\-]?\s*)(?:₹|rs\.?\s*)\s*([0-9,]+(\.\d{1,2})?)",full); g["Premium Amount"]="₹"+m.group(2) if m else "—"
#     m=re.search(r"(sum\s*assured)\s*[:\-]?\s*(?:₹|rs\.?\s*)?\s*([0-9,]+)",full); g["Sum Assured"]="₹"+m.group(2) if m else "—"
#     m=re.search(r"(fund\s*value)\s*[:\-]?\s*(?:₹|rs\.?\s*)?\s*([0-9,]+(\.\d{1,2})?)",full); g["Fund Value"]="₹"+m.group(2) if m else "—"
#     m=re.search(r"(policy\s*status)\s*[:\-]?\s*(active|inactive|lapsed|in force|paid up)",full); g["Policy Status"]=m.group(2).title() if m else "—"
#     m=re.search(r"(payment\s*(mode|method))\s*[:\-]?\s*(cash|card|netbanking|upi|google\s*pay|gpay|phonepe|paytm)",full)
#     g["Payment Mode"]=(m.group(3).title().replace("Gpay","GPay").replace("Google Pay","Google Pay").replace("Phonepe","PhonePe")) if m else "—"
#     return g

# # -----------------------
# # Report (HTML) — auto-rendered
# # -----------------------
# def build_report_html(fields: dict, title: str = "BRU - CQ Sheet") -> str:
#     def fv(k): return escape(str(fields.get(k,"—") or "—"))
#     style = """
#     <style>
#       .vs-wrap{max-width:1100px;margin:10px auto 24px;font-family:ui-sans-serif,system-ui,Segoe UI,Roboto,Helvetica,Arial}
#       .vs-card{background:#111319;border:1px solid #2a2f3a;border-radius:14px;padding:16px;color:#e7e9ee}
#       .vs-title{font-size:20px;font-weight:800;margin:0 0 12px;color:#f2f4f8}
#       .meta{width:100%;border-collapse:collapse;font-size:13px}
#       .meta th,.meta td{border:1px solid #2a2f3a;padding:8px 10px}
#       .meta th{background:#0c0e13;text-align:left;width:20%;color:#b8c1ce}
#       .meta td{background:#0c0f16}
#       .sec{margin-top:16px}
#       .grid{width:100%;border-collapse:separate;border-spacing:0 6px;font-size:13px}
#       .grid th{background:#0c0e13;color:#aeb6c2;text-align:left;padding:10px;border:1px solid #2a2f3a;border-right:none;border-radius:8px 0 0 8px;width:35%}
#       .grid td{background:#0c0f16;padding:10px;border:1px solid #2a2f3a;border-left:none;border-radius:0 8px 8px 0}
#       .muted{color:#9aa3af}
#     </style>
#     """
#     meta = f"""
#     <table class="meta">
#       <tr><th>Policy No</th><td>{fv('Policy No')}</td><th>Associate Name</th><td>{fv('Associate Name')}</td></tr>
#       <tr><th>Customer Name</th><td>{fv('Customer Name')}</td><th>Mobile Number</th><td>{fv('Mobile Number')}</td></tr>
#       <tr><th>Due Date</th><td>{fv('Due Date')}</td><th>Premium Amount</th><td>{fv('Premium Amount')}</td></tr>
#       <tr><th>Sum Assured</th><td>{fv('Sum Assured')}</td><th>Fund Value</th><td>{fv('Fund Value')}</td></tr>
#       <tr><th>Policy Status</th><td>{fv('Policy Status')}</td><th>Payment Mode</th><td>{fv('Payment Mode')}</td></tr>
#     </table>
#     """

#     rows = [
#         ("Opening a Call","Greeting, self-introduction, branding & opening."),
#         ("Opening a Call","Ask for customer’s availability to start the conversation."),
#         ("Customer Service & Assistance","Linkage with last recorded disposition/remarks."),
#         ("Customer Service & Assistance","Identify need & provide suitable response."),
#         ("Retention & Knowledge","Explore reasons for non-payment; educate benefits/charges."),
#         ("Compliance","Inform correct premium & due date."),
#         ("Soft Skill & Communication","Active listening; quick responses."),
#         ("Call Closing","Close as per script."),
#         ("Data Enrichment","Collect email & alternate number."),
#         ("Call Score","(Template row; scoring wiring TBD)"),
#     ]
#     grid = "".join(f"<tr><th>{escape(sec)}</th><td>{escape(item)}</td></tr>" for sec,item in rows)

#     return f"""{style}
#     <div class="vs-wrap">
#       <div class="vs-card">
#         <div class="vs-title">{escape(title)}</div>
#         {meta}
#         <div class="sec"><table class="grid">{grid}</table>
#         </div>
#       </div>
#     </div>
#     """

# def generate_report_html_from_state(dialogue, fields):
#     try:
#         if not dialogue:
#             return ("<div class='vs-wrap'><div class='vs-card'><div class='vs-title'>BRU - CQ Sheet</div>"
#                     "<div class='muted'>⚠️ Process an audio first to build the report.</div></div></div>")
#         if not fields or not isinstance(fields, dict):
#             fields = extract_fields_from_dialogue(dialogue)
#         return build_report_html(fields, title="BRU - CQ Sheet")
#     except Exception as e:
#         return f"<div class='vs-wrap'><div class='vs-card'><div class='vs-title'>BRU - CQ Sheet</div><div class='muted'>Error while building report: {escape(str(e))}</div></div></div>"

# # -----------------------
# # Main callback — returns the report HTML directly (no .then needed)
# # -----------------------
# def transcribe_and_diarize(audio_file, model_name, enable_diarization, hf_token):
#     logs=[]
#     if audio_file is None:
#         html = "<div class='vs-wrap'><div class='vs-card'><div class='vs-title'>BRU - CQ Sheet</div><div class='muted'>Upload an audio first.</div></div></div>"
#         return "_Please upload an audio file._", "{}", "No audio provided.", [], {}, html

#     input_path = audio_file
#     cleaned_path = os.path.join(os.path.dirname(input_path), "cleaned_audio_for_asr_and_diarization.wav")

#     ok = smart_audio_preprocessing(input_path, cleaned_path, logs)
#     if not ok:
#         html = "<div class='vs-wrap'><div class='vs-card'><div class='vs-title'>BRU - CQ Sheet</div><div class='muted'>Preprocessing failed.</div></div></div>"
#         return "_Preprocessing failed._", "{}", "\n".join(logs), [], {}, html

#     try:
#         whisper_result = enhanced_whisper_transcription(cleaned_path, model_name, logs)
#     except Exception as e:
#         logs.append(f"❌ Whisper transcription failed: {e}")
#         html = f"<div class='vs-wrap'><div class='vs-card'><div class='vs-title'>BRU - CQ Sheet</div><div class='muted'>Transcription failed: {escape(str(e))}</div></div></div>"
#         return "_Transcription failed._", "{}", "\n".join(logs), [], {}, html

#     segments = (whisper_result.get("segments") or [])
#     cleaned_segments = detect_and_remove_repetitions(segments, log=logs)

#     processed_segments = []
#     for seg in cleaned_segments:
#         processed_text = post_process_text(seg['text'])
#         if processed_text.strip() and len(processed_text.strip()) > 5:
#             seg_copy = dict(seg); seg_copy['text'] = processed_text
#             processed_segments.append(seg_copy)

#     diarization = run_diarization(cleaned_path, hf_token, logs) if enable_diarization else None

#     dialogue = build_dialogue(processed_segments, diarization)
#     fields   = extract_fields_from_dialogue(dialogue)

#     md = format_dialogue_markdown(dialogue)
#     payload = {
#         "metadata": {
#             "total_duration": whisper_result.get('duration', 0),
#             "total_speakers": len(set(d['speaker'] for d in dialogue)),
#             "total_segments": len(dialogue),
#             "model_used": f"whisper-{model_name}",
#             "processing_successful": True,
#             "anti_repetition_applied": True,
#             "diarization_enabled": bool(enable_diarization),
#         },
#         "dialogue": dialogue,
#         "fields": fields,
#     }

#     # Build the report HTML HERE and return it as an output
#     report_html = generate_report_html_from_state(dialogue, fields)
#     return md, json.dumps(payload, indent=2, ensure_ascii=False), "\n".join(logs), dialogue, fields, report_html

# # -----------------------
# # UI
# # -----------------------
# latest_dialogue_state = gr.State([])
# latest_fields_state  = gr.State({})

# with gr.Blocks(title="Speech Analysis - QA") as demo:
#     gr.Markdown("# 🗣️ Whisper Call Transcriber → 📑 Report\nUpload a call recording, get a clean dialogue and a report like your reference sheet.")

#     with gr.Row():
#         audio = gr.Audio(type="filepath", label="Upload audio (wav/mp3/m4a...)")
#     with gr.Row():
#         model_name = gr.Dropdown(choices=["large-v3","large","medium","small","base","tiny"],
#                                  value=DEFAULT_MODEL_NAME, label="Whisper model")
#         enable_diar = gr.Checkbox(value=True, label="Enable diarization (Pyannote)")
#         hf_token = gr.Textbox(value=DEFAULT_HF_TOKEN,
#                               label="Hugging Face token (required if diarization enabled)",
#                               type="password", placeholder="hf_xxx...")

#     run_btn = gr.Button("▶️ Process")

#     with gr.Tab("Dialogue"):
#         dialogue_md = gr.Markdown()

#     with gr.Tab("Report Analysis"):
#         gr.Markdown("Report generated automatically from the audio-derived dialogue.")
#         report_html = gr.HTML(value="<div class='vs-wrap'><div class='vs-card'><div class='vs-title'>BRU - CQ Sheet</div><div class='muted'>_No report yet._</div></div></div>")

#     with gr.Tab("JSON"):
#         json_out = gr.Code(language="json", label="Output JSON")

#     with gr.Tab("Logs"):
#         logs_out = gr.Textbox(label="Processing Logs", lines=18)
#         dialogue_inline = gr.Markdown(value="_Dialogue will appear here_")

#     # Single callback returns EVERYTHING, including the report HTML (no .then chain for report)
#     run_btn.click(
#         transcribe_and_diarize,
#         inputs=[audio, model_name, enable_diar, hf_token],
#         outputs=[dialogue_md, json_out, logs_out, latest_dialogue_state, latest_fields_state, report_html]
#     ).then(
#         lambda md, *_: md, inputs=[dialogue_md], outputs=[dialogue_inline]
#     )

# if __name__ == "__main__":
#     gr.close_all()
#     demo.launch(share=True)


In [ ]:
#part - 1 for report structure

In [3]:
# app_gradio.py
# ---------------------------------------------------------
# Whisper + (optional) Pyannote UI with "Dialogue" and "Report Analysis"
# ---------------------------------------------------------

import os, re, json, subprocess
from functools import lru_cache
from typing import List, Dict, Any
from html import escape
from datetime import datetime

import gradio as gr
import torch
import whisper
from pyannote.audio import Pipeline

# -----------------------
# Config
# -----------------------
DEFAULT_MODEL_NAME = os.getenv("WHISPER_MODEL", "large-v3")
SAMPLE_RATE_HZ = 16000
DEFAULT_HF_TOKEN = os.getenv("HUGGING_FACE_TOKEN", "hf_")

# NEW: default owners for every audit
DEFAULT_TL_NAME = "Shalini"
DEFAULT_QA_NAME = "Kalaivani"

#agents names list
KNOWN_AGENTS = [
    "Jaya Parkash", "Chandru", "Sneha", "Kowsalya", "Swathi",
    "Arogya Marry", "Delphina", "Aesu Marry"
]

SCORE_CHOICES = ["Yes", "No", "N/A"]

# -----------------------
# Date function for Audit Date
# -----------------------
def mdy_from_file(path: str) -> str:
    try:
        ts = os.path.getmtime(path)
        dt = datetime.fromtimestamp(ts)
        # M/D/YYYY (no leading zeros)
        return f"{dt.month}/{dt.day}/{dt.year}"
    except Exception:
        return ""

# -----------------------
# FFmpeg / audio utils
# -----------------------
def get_audio_duration(audio_path: str) -> float:
    try:
        out = subprocess.run(
            ["ffprobe","-v","error","-show_entries","format=duration","-of","default=noprint_wrappers=1:nokey=1",audio_path],
            capture_output=True, text=True, check=True
        )
        return float(out.stdout.strip())
    except Exception:
        return 0.0

def _ffmpeg_run(cmd: List[str]) -> bool:
    try:
        subprocess.run(cmd, check=True, capture_output=True, text=True)
        return True
    except subprocess.CalledProcessError:
        return False

def audio_preprocessing_v1(i,o):
    return _ffmpeg_run(["ffmpeg","-i",i,"-acodec","pcm_s16le","-ac","1","-ar",str(SAMPLE_RATE_HZ),
                        "-af","loudnorm=I=-23:TP=-2:LRA=7,highpass=f=80,lowpass=f=8000,afftdn=nr=10","-y",o])
def audio_preprocessing_v2(i,o):
    return _ffmpeg_run(["ffmpeg","-i",i,"-acodec","pcm_s16le","-ac","1","-ar",str(SAMPLE_RATE_HZ),
                        "-af","loudnorm=I=-23:TP=-2,highpass=f=100","-y",o])
def audio_preprocessing_v3(i,o):
    return _ffmpeg_run(["ffmpeg","-i",i,"-acodec","pcm_s16le","-ac","1","-ar",str(SAMPLE_RATE_HZ),
                        "-af","loudnorm","-y",o])
def audio_preprocessing_v4(i,o):
    return _ffmpeg_run(["ffmpeg","-i",i,"-acodec","pcm_s16le","-ac","1","-ar",str(SAMPLE_RATE_HZ),"-y",o])

def smart_audio_preprocessing(input_path, output_path, log: List[str]) -> bool:
    original_dur = get_audio_duration(input_path)
    log.append(f"Original audio duration: {original_dur:.2f}s")
    methods = [("Advanced (afftdn)",audio_preprocessing_v1),("Simplified",audio_preprocessing_v2),
               ("Basic",audio_preprocessing_v3),("Minimal",audio_preprocessing_v4)]
    for idx,(name,fn) in enumerate(methods,1):
        log.append(f"→ Trying {name} preprocessing...")
        if fn(input_path, output_path) and os.path.exists(output_path):
            processed_dur = get_audio_duration(output_path)
            log.append(f"Processed duration: {processed_dur:.2f}s")
            if abs(original_dur - processed_dur) < 1.0:
                log.append(f"✅ Preprocessing successful with method {idx}: {name}")
                return True
            else:
                log.append(f"⚠️ Duration mismatch on method {idx}; trying next...")
    log.append("❌ All preprocessing methods failed.")
    return False

# -----------------------
# Text cleanup / repetition control
# -----------------------
def post_process_text(text: str) -> str:
    if not text: return ""
    words = text.split()
    cleaned_words, i = [], 0
    while i < len(words):
        cur = words[i].lower(); count = 1; j = i+1
        while j < len(words) and words[j].lower() == cur: count += 1; j += 1
        cleaned_words.extend([words[i]] * (min(count,2) if count <= 3 else 1)); i += count
    text = " ".join(cleaned_words)

    filler = ["uh","um","mm","hmm","ah","oh","huh","ha ha",
              "okay okay","yes yes","yes yes yes","i mean","you know","like like","ok ok"]
    for f in filler:
        text = re.sub(rf'\b{re.escape(f)}\b','',text,flags=re.IGNORECASE)

    repl = {'access max life':'Axis Max Life','axis max life':'Axis Max Life','g pay':'GPay',
            'google pay':'Google Pay','phone pay':'PhonePe','phone pe':'PhonePe','pay tm':'Paytm',
            'net banking':'netbanking','some assured':'sum assured','premium do':'premium due',
            'do date':'due date','okay sir':'Okay sir'}
    low = text.lower()
    for w,c in repl.items(): low = low.replace(w,c)
    low = re.sub(r'\brs[.]?\s*','₹',low)
    low = re.sub(r'\s{2,}',' ',low)
    low = re.sub(r'[,]{2,}',',',low)
    low = re.sub(r'\s+,',',',low)
    low = re.sub(r'\s+\.','.',low)
    low = re.sub(r'\s+[!?]', lambda m:m.group(0).strip(), low)
    low = re.sub(r'(^|[.!?]\s+)([a-z])', lambda m:m.group(1)+m.group(2).upper(), low)
    return low.strip()

def detect_and_remove_repetitions(segments, max_repetition_ratio=0.3, log=None):
    log = log or []; log.append("🔍 Aggressive repetition detection started...")
    cleaned = []
    for seg in segments:
        text = seg["text"].strip(); words = text.split()
        if len(words) < 2: continue
        wc = {}
        for w in words:
            wl = w.lower().strip('.,!?'); wc[wl] = wc.get(wl,0)+1
        dominance = (max(wc.values())/len(words)) if wc else 0
        if dominance > 0.4: log.append(f"🚫 Drop (dominant word): {text[:50]}..."); continue

        consec=max_consec=0
        for j in range(1,len(words)):
            if words[j].lower().strip('.,!?') == words[j-1].lower().strip('.,!?'):
                consec += 1; max_consec = max(max_consec, consec+1)
            else: consec = 0
        if max_consec > 3: log.append(f"🚫 Drop (consecutive repeats): {text[:50]}..."); continue

        repetitive=False
        for L in range(2, min(len(words)//3+1,8)):
            for s in range(len(words)-L*2+1):
                if ' '.join(words[s:s+L]).lower() == ' '.join(words[s+L:s+2*L]).lower():
                    if (L*2)/len(words) > max_repetition_ratio:
                        log.append(f"🚫 Drop (pattern repeat): {text[:50]}..."); repetitive=True; break
            if repetitive: break
        if repetitive: continue

        is_dup=False
        for prev in cleaned[-5:]:
            a,b=set(prev['text'].lower().split()), set(w.lower() for w in words)
            if a|b and (len(a&b)/len(a|b))>0.7 and abs(len(a)-len(b))<5:
                log.append(f"🚫 Drop (near-duplicate): {text[:50]}..."); is_dup=True; break
        if is_dup: continue
        cleaned.append(seg)
    log.append(f"📊 Cleaned {len(segments)} → {len(cleaned)} segments")
    return cleaned

# -----------------------
# Whisper + diarization
# -----------------------
@lru_cache(maxsize=1)
def get_whisper_model(name: str):
    return whisper.load_model(name)

def enhanced_whisper_transcription(audio_path: str, model_name: str, log) -> Dict[str, Any]:
    log.append("--- Enhanced Whisper Transcription ---")
    agents_str = ", ".join(KNOWN_AGENTS)
    prompt = (
        f"This is a customer support call from Axis Maxlife Insurance in Tamil regarding insurance renewal call. "
        f"Agents are always one of these: {agents_str}. Use exact names like 'Jaya Parkash' or 'Swathi'. "
        f"Discuss policy numbers, due dates, fund values, sum assured, late fees, and payment methods such as "
        f"Google Pay, PhonePe, Paytm, netbanking."
    )
    model = get_whisper_model(model_name)
    result = model.transcribe(
        audio_path, language="ta", task="translate", temperature=0.0,
        beam_size=5, patience=1.2, condition_on_previous_text=False,
        no_speech_threshold=0.8, compression_ratio_threshold=2.0,
        logprob_threshold=-0.35, word_timestamps=False,
        initial_prompt=prompt, verbose=True,
    )
    log.append("✅ Whisper transcription done.")
    return result

def run_diarization(wav_path: str, hf_token: str, log):
    try:
        token = hf_token or os.getenv("HUGGING_FACE_TOKEN", "")
        if not token:
            log.append("ℹ️ No HF token provided: skipping diarization (Speaker_Unknown).")
            return None
        try:
            pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=token)
        except TypeError:
            pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", token=token)
        if torch.cuda.is_available():
            pipeline.to(torch.device("cuda")); log.append("✅ Using GPU for diarization.")
        diarization = pipeline(wav_path); log.append("✅ Speaker diarization completed.")
        return diarization
    except Exception as e:
        log.append(f"⚠️ Diarization failed: {e}"); return None

# -----------------------
# Dialogue + fields
# -----------------------
def get_dominant_speaker(start_time, end_time, diarization_result):
    if not diarization_result: return "Speaker_Unknown"
    speakers = {}
    for segment, _, speaker in diarization_result.itertracks(yield_label=True):
        overlap_start = max(start_time, segment.start)
        overlap_end = min(end_time, segment.end)
        dur = max(0.0, overlap_end - overlap_start)
        if dur > 0: speakers[speaker] = speakers.get(speaker, 0.0) + dur
    return max(speakers, key=speakers.get) if speakers else "Speaker_Unknown"

def build_dialogue(processed_segments, diarization):
    out=[]; cur_spk=None; cur_texts=[]; cur_start=0.0; cur_end=0.0
    for seg in processed_segments:
        start,end,txt = seg['start'], seg['end'], seg['text'].strip()
        spk = get_dominant_speaker(start,end,diarization)
        if (spk==cur_spk and cur_spk and (start-cur_end)<3.0):
            cur_texts.append(txt); cur_end=end
        else:
            if cur_spk and cur_texts:
                c=' '.join(cur_texts)
                if len(c.strip())>10: out.append({'speaker':cur_spk,'text':c,'start_time':cur_start,'end_time':cur_end})
            cur_spk,cur_texts,cur_start,cur_end = spk,[txt],start,end
    if cur_spk and cur_texts:
        c=' '.join(cur_texts)
        if len(c.strip())>10: out.append({'speaker':cur_spk,'text':c,'start_time':cur_start,'end_time':cur_end})
    return out

def format_dialogue_markdown(dialogue: List[Dict[str, Any]]) -> str:
    if not dialogue: return "_No substantial dialogue detected._"
    lines=["## 🎭 Dialogue",""]
    for d in dialogue:
        lines.append(f"**{d['speaker']} [{d['start_time']:.1f}s - {d['end_time']:.1f}s]:**")
        lines.append(d['text']); lines.append("")
    return "\n".join(lines)

def _norm(t: str) -> str: return re.sub(r"\s+"," ",t).strip()

PLAN_HINT_WORDS = ["plan", "policy", "smart", "wealth", "lump", "limited", "pay", "non", "par", "ulip", "term", "life", "max"]
KNOWN_COHORTS = {"FRYP","NRYP","RYP","FY","REN","FR","FRY","FRYP1","FRYP2"}
KNOWN_CHANNELS = {"UCB","DCB","RBS","INBOUND","OUTBOUND","BRU","BRM","BRL"}
KNOWN_DISPO = {
    "call back":"Call Back", "callback":"Call Back", "cb":"Call Back",
    "not reachable":"Not Reachable", "nr":"Not Reachable",
    "rpc":"RPC", "cb-rpc":"CB-RPC", "cb rpc":"CB-RPC",
    "paid":"Paid", "promise to pay":"Promise To Pay", "ptp":"Promise To Pay"
}
KNOWN_SUBDISPO = {
    "cb-rpc":"CB-RPC", "rpc":"RPC", "na":"NA", "not interested":"Not Interested",
    "wrong number":"Wrong Number", "follow up":"Follow Up", "payment link":"Payment Link Sent"
}

AGENT_STOP_RE = r"(?:from|calling|call|here|with|team|branch|,|\.|$)"

def find_associate_name(full_txt: str) -> str:
    # 1) Prefer an exact match from the known agent list
    low = full_txt.lower()
    for nm in KNOWN_AGENTS:
        if nm.lower() in low:
            return nm  # keep canonical casing from the list

    # 2) Natural phrases like "my name is <name>" / "this is <name>" / "I am <name>"
    m = re.search(
        rf"(?:my\s+name\s+is|this\s+is|i\s*am|speaking)\s+([A-Za-z][A-Za-z.\s]{{1,30}}?)(?=\s+{AGENT_STOP_RE})",
        full_txt, flags=re.I
    )
    if m:
        return _norm(m.group(1).title())

    # 3) "Associate/Agent name: <name>"
    m = re.search(r"(?:associate|agent)\s*name\s*[:\-]?\s*([A-Za-z][A-Za-z.\s]{1,30})", full_txt, flags=re.I)
    if m:
        return _norm(m.group(1).title())

    return "—"



def extract_fields_from_dialogue(dialogue):
    full_txt = " ".join(_norm(d["text"]) for d in dialogue)
    full = full_txt.lower()
    g = {}

    # Policy Number (allow "policy number is 149607129")
    m = re.search(r"policy\s*(?:no|number)\s*(?:is|:|-)?\s*(\d{6,16})", full)
    if not m:
        # fallback: "... number 149607129 for your policy"
        m = re.search(r"\b(\d{6,16})\b(?=.*\bpolicy\s*(?:no|number)\b)", full)
    g["Policy No"] = m.group(1) if m else "—"

    # Associate (agent) name
    g["Associate Name"] = find_associate_name(full_txt)

    # Customer name (Mr./Mrs./Ms. …)
    m = re.search(r"(?:mr|mrs|ms)\.?\s+([a-zA-Z][a-zA-Z.\s]{2,30})", full)
    g["Customer Name"] = m.group(1).title().strip() if m else "—"

    # Mobile
    m = re.search(r"(?:mobile|phone|contact)\s*(?:no|number)?\s*[:\-]?\s*(\+?91[\s\-]?)?(\d{10})", full)
    g["Mobile Number"] = (("+91 " if m and m.group(1) else "") + (m.group(2) if m else "—")) if m else "—"

    # Due date
    m = re.search(r"(?:due\s*date|premium\s*due)\s*[:\-]?\s*([0-3]?\d[\/\-][01]?\d(?:[\/\-]\d{2,4})?|[0-3]?\d\s+[a-zA-Z]{3,9}\s+\d{4})", full)
    g["Due Date"] = m.group(1) if m else "—"

    # Premium / Sum Assured / Fund
    m = re.search(r"(?:premium\s*(?:amount|due)?\s*[:\-]?\s*)(?:₹|rs\.?\s*)\s*([0-9,]+(?:\.\d{1,2})?)", full)
    g["Premium Amount"] = "₹"+m.group(1) if m else "—"

    m = re.search(r"(?:sum\s*assured)\s*[:\-]?\s*(?:₹|rs\.?\s*)?\s*([0-9,]+)", full)
    g["Sum Assured"] = "₹"+m.group(1) if m else "—"

    m = re.search(r"(?:fund\s*value)\s*[:\-]?\s*(?:₹|rs\.?\s*)?\s*([0-9,]+(?:\.\d{1,2})?)", full)
    g["Fund Value"] = "₹"+m.group(1) if m else "—"

    # Policy Status / Payment Mode
    m = re.search(r"(?:policy\s*status)\s*[:\-]?\s*(active|inactive|lapsed|in force|paid up)", full)
    g["Policy Status"] = m.group(1).title() if m else "—"

    m = re.search(r"(?:payment\s*(?:mode|method))\s*[:\-]?\s*(cash|card|netbanking|upi|google\s*pay|gpay|phonepe|paytm)", full)
    g["Payment Mode"] = (m.group(1).title().replace("Gpay","GPay").replace("Google Pay","Google Pay").replace("Phonepe","PhonePe")) if m else "—"

    # --- Plan Name (same robust logic you had) ---
    m = re.search(r"(?:plan\s*name|policy\s*plan)\s*[:\-]?\s*([A-Za-z0-9 ,&\-/]+plan(?:\s*\([^)]+\))?)", full_txt, flags=re.I)
    plan = m.group(1).strip() if m else ""
    if not plan:
        m = re.search(r"(Max\s*Life[^\n,;.]{6,120}?Plan(?:\s*\([^)]+\))?)", full_txt, flags=re.I)
        if m: plan = _norm(m.group(1))
    if not plan:
        m = re.search(r"([A-Za-z][A-Za-z0-9 ,&\-/]{8,120}?plan(?:\s*\([^)]+\))?)", full_txt, flags=re.I)
        if m and any(w in m.group(1).lower() for w in PLAN_HINT_WORDS):
            plan = _norm(m.group(1))
    g["Plan Name"] = plan if plan else "—"

    # --- Cohort / Channel / Disposition / Sub-Disposition ---
    up = full_txt.upper()
    g["Cohort"] = next((c for c in KNOWN_COHORTS if re.search(rf"\b{re.escape(c)}\b", up)), "—")
    g["Channel Name"] = next((c for c in KNOWN_CHANNELS if re.search(rf"\b{re.escape(c)}\b", up)), "—")

    dispo = "—"
    for k,v in KNOWN_DISPO.items():
        if re.search(rf"\b{k}\b", full):
            dispo = v; break
    g["Disposition"] = dispo

    subd = "—"
    for k,v in KNOWN_SUBDISPO.items():
        if re.search(rf"\b{k}\b", full):
            subd = v; break
    g["Sub-Disposition"] = subd

    return g

# -----------------------
# Scorecard (QA) — spec & HTML
# -----------------------
QUALITY_ITEMS = [
    # Opening a Call
    {"group":"Opening a Call","sn":1, "text":"Greeting, Self-introduction, Branding & Opening.", "fatality":"Non Fatal", "weight":3},
    {"group":"Opening a Call","sn":2, "text":"Ask for customer's availability to start the conversation.", "fatality":"Non Fatal", "weight":2},

    # Customer Service & Assistance
    {"group":"Customer Service & Assistance","sn":3, "text":"Linkage of current communication with last recorded conversation/disposition code with customer.", "fatality":"Non Fatal", "weight":5},
    {"group":"Customer Service & Assistance","sn":4, "text":"Understanding/probing customer's need to provide suitable response for collection & retention.", "fatality":"Non Fatal", "weight":6},
    {"group":"Customer Service & Assistance","sn":5, "text":"Hold and Mute procedure and dead air.", "fatality":"Non Fatal", "weight":6},
    {"group":"Customer Service & Assistance","sn":6, "text":"Ask if they need any assistance in making the payment & advise on appropriate payment option / ECS Registration / Transaction Status, Draw Date & Hit Date (if applicable).", "fatality":"Fatal", "weight":6},

    # Retention & Knowledge
    {"group":"Retention & Knowledge","sn":7, "text":"Pitch product features (USPs) and persuade the customer to create interest.", "fatality":"Non Fatal", "weight":6},
    {"group":"Retention & Knowledge","sn":8, "text":"Rebuttals for RTP / deferred payment / misselling / low fund value / financial constraint / Payment & Policy Term.", "fatality":"Fatal", "weight":10},

    # Compliance
    {"group":"Compliance","sn":9, "text":"Inform correct C2P / late fee & due date / Sum assured / maturity & surrender amount.", "fatality":"Non Fatal", "weight":10},
    {"group":"Compliance","sn":10, "text":"Should not disclose customer's personal information on call to third party.", "fatality":"Fatal", "weight":10},

    # Soft Skill & Communication
    {"group":"Soft Skill & Communication","sn":11, "text":"Active listening & quick response.", "fatality":"Non Fatal", "weight":6},
    {"group":"Soft Skill & Communication","sn":12, "text":"No rude / negative statement on the call.", "fatality":"Fatal", "weight":10},
    {"group":"Soft Skill & Communication","sn":13, "text":"Soft skill.", "fatality":"Non Fatal", "weight":4},

    # Call Closing
    {"group":"Call Closing","sn":14, "text":"Call closing as per script.", "fatality":"Non Fatal", "weight":4},

    # Calling Disposition / Data Enrichment
    {"group":"Calling Disposition / Data Enrichment","sn":15, "text":"Alter number and Email ID (if any) and correct follow-up dates.", "fatality":"Non Fatal", "weight":2},
    {"group":"Calling Disposition / Data Enrichment","sn":16, "text":"Dispose the call correct/Short Codes & ensure no repeat calls/unnecessary callback.", "fatality":"Non Fatal", "weight":10},

]

def _build_scorecard_html():
    # Build rows with group rowspans
    from collections import defaultdict
    group_counts = defaultdict(int)
    for item in QUALITY_ITEMS:
        group_counts[item["group"]] += 1

    weight_total = sum(item["weight"] for item in QUALITY_ITEMS)
    # If later you set item["score"]="Yes"/"No", these totals will auto-calc:
    scored_total = 0
    fatal_errors = 0
    for item in QUALITY_ITEMS:
        score = item.get("score", "—")  # "Yes"/"No"/"—"
        if score == "Yes":
            scored_total += item["weight"]
        elif score == "No" and item["fatality"].lower() == "fatal":
            fatal_errors += 1

    final_percent = f"{round((scored_total/weight_total)*100)}%" if weight_total else "—"

    # Styles for scorecard (shares palette with your sheet)
    style = """
    <style>
      .qa-card{margin-top:14px}
      .qa-table{width:100%;border-collapse:collapse;font-size:13px}
      .qa-table th,.qa-table td{border:1px solid #2a2f3a;padding:8px 10px;vertical-align:top}
      .qa-head{background:#0b1220;color:#f5f7fb;text-align:center;font-weight:800;font-size:15px}
      .qa-col-h{background:#0c1427;color:#b9c3d4;text-align:left;font-weight:700}
      .qa-group{background:#0c1427;color:#b9c3d4;font-weight:700;min-width:180px}
      .qa-cell{background:#0c0f16;color:#e7e9ee}
      .qa-right{text-align:right}
      .qa-center{text-align:center}
      .qa-sum{background:#0c0f16;color:#e7e9ee;font-weight:700}
    </style>
    """

    # Header
    head = """
    <table class="qa-table">
      <tr><th class="qa-head" colspan="7">Call Score</th></tr>
      <tr>
        <th class="qa-col-h">Parameters</th>
        <th class="qa-col-h qa-center">S.no.</th>
        <th class="qa-col-h">Sub-Parameters</th>
        <th class="qa-col-h">Fatality</th>
        <th class="qa-col-h qa-center">Score</th>
        <th class="qa-col-h qa-right">Weightage</th>
        <th class="qa-col-h qa-right">Final Score</th>
      </tr>
    """

    # Body rows with rowspans for group
    body = []
    seen = defaultdict(int)
    for item in QUALITY_ITEMS:
        group = item["group"]
        sn = item["sn"]
        text = item["text"]
        fatal = item["fatality"]
        score = item.get("score", "—")
        weight = item["weight"]
        final = (weight if score == "Yes" else 0) if score in ("Yes", "No") else "—"

        tds = []
        if seen[group] == 0:
            rowspan = group_counts[group]
            tds.append(f'<td class="qa-group" rowspan="{rowspan}">{escape(group)}</td>')
        seen[group] += 1

        tds.extend([
            f'<td class="qa-cell qa-center">{sn}</td>',
            f'<td class="qa-cell">{escape(text)}</td>',
            f'<td class="qa-cell">{escape(fatal)}</td>',
            f'<td class="qa-cell qa-center">{escape(str(score))}</td>',
            f'<td class="qa-cell qa-right">{weight}</td>',
            f'<td class="qa-cell qa-right">{final}</td>',
        ])
        body.append("<tr>" + "".join(tds) + "</tr>")

    # Summary rows
    summary = f"""
      <tr>
        <td class="qa-col-h" colspan="5">Count of Fatal Error</td>
        <td class="qa-sum qa-right" colspan="2">{fatal_errors}</td>
      </tr>
      <tr>
        <td class="qa-col-h" colspan="5">Total Score including Fatal Error</td>
        <td class="qa-sum qa-right">{scored_total}</td>
        <td class="qa-sum qa-right">{'100%' if weight_total else '—'}</td>
      </tr>
      <tr>
        <td class="qa-col-h" colspan="5">Final Score</td>
        <td class="qa-sum qa-right" colspan="2">{final_percent}</td>
      </tr>
    </table>
    """

    return f'<div class="qa-card">{style}{head}{"".join(body)}{summary}</div>'


def _apply_scores_to_items(scores):
    # mutate QUALITY_ITEMS in-place with the selected scores
    for i, s in enumerate(scores):
        if i < len(QUALITY_ITEMS) and s in SCORE_CHOICES:
            QUALITY_ITEMS[i]["score"] = s

def update_scorecard_html(*vals):
    # last arg is the fields dict from latest_fields_state (Gradio State)
    *scores, fields = vals
    _apply_scores_to_items(scores)
    return build_report_html(fields, title="BRU - CQ Sheet")

# -----------------------
# Report (HTML) — auto-rendered
# -----------------------
def build_report_html(fields: dict, title: str = "BRU - CQ Sheet") -> str:
    from html import escape

    wanted_order = [
        "Policy No",
        "Associate Name",   # (agent name)
        "TL Name",
        "Plan Name",
        "QA Name",
        "Audit Date",
        "Cohort",
        "Channel Name",
        "Disposition",
        "Sub-Disposition",
    ]

    def fv(k):
        v = fields.get(k, "—")
        return escape(str(v if v not in (None, "", []) else "—"))

    title = escape(title)

    style = """
    <style>
      .vs-wrap{max-width:1000px;margin:10px auto 24px;font-family:ui-sans-serif,system-ui,Segoe UI,Roboto,Helvetica,Arial}
      .vs-card{background:#111319;border:1px solid #2a2f3a;border-radius:14px;padding:0;color:#e7e9ee;overflow:hidden}
      .sheet{width:100%;border-collapse:collapse;font-size:14px}
      .sheet th,.sheet td{border:1px solid #2a2f3a;padding:10px 12px}
      .hdr{background:#0b1220;color:#f5f7fb;text-align:left;font-weight:800;font-size:16px;letter-spacing:.2px;padding:10px 12px}
      .label{background:#0c1427;color:#b9c3d4;width:34%;font-weight:600;text-align:left}
      .value{background:#0c0f16;color:#e7e9ee;text-align:left}
      .spacer{height:12px}
    </style>
    """

    # 10-field sheet (your existing part)
    rows_html = [f"<tr><th colspan='2' class='hdr'>{title}</th></tr>"]
    for key in wanted_order:
        rows_html.append(f"<tr><td class='label'>{escape(key)}</td><td class='value'>{fv(key)}</td></tr>")
    sheet_html = f"<table class='sheet'>{''.join(rows_html)}</table>"

    # Append the scorecard block
    scorecard_html = _build_scorecard_html()

    return f"""{style}
    <div class="vs-wrap">
      <div class="vs-card">
        {sheet_html}
        <div class="spacer"></div>
        {scorecard_html}
      </div>
    </div>
    """



def generate_report_html_from_state(dialogue, fields):
    try:
        if not dialogue:
            return ("<div class='vs-wrap'><div class='vs-card'><div class='vs-title'>BRU - CQ Sheet</div>"
                    "<div class='muted'>⚠️ Process an audio first to build the report.</div></div></div>")
        if not fields or not isinstance(fields, dict):
            fields = extract_fields_from_dialogue(dialogue)
        return build_report_html(fields, title="BRU - CQ Sheet")
    except Exception as e:
        return f"<div class='vs-wrap'><div class='vs-card'><div class='vs-title'>BRU - CQ Sheet</div><div class='muted'>Error while building report: {escape(str(e))}</div></div></div>"

# -----------------------
# Main callback — returns the report HTML directly (no .then needed)
# -----------------------
def transcribe_and_diarize(audio_file, model_name, enable_diarization, hf_token):
    logs=[]
    if audio_file is None:
        html = "<div class='vs-wrap'><div class='vs-card'><div class='vs-title'>BRU - CQ Sheet</div><div class='muted'>Upload an audio first.</div></div></div>"
        return "_Please upload an audio file._", "{}", "No audio provided.", [], {}, html

    input_path = audio_file
    cleaned_path = os.path.join(os.path.dirname(input_path), "cleaned_audio_for_asr_and_diarization.wav")
    # Compute Audit Date from the uploaded file (fallback to cleaned file if needed)
    audit_date_str = mdy_from_file(input_path) or mdy_from_file(cleaned_path)

    ok = smart_audio_preprocessing(input_path, cleaned_path, logs)
    if not ok:
        html = "<div class='vs-wrap'><div class='vs-card'><div class='vs-title'>BRU - CQ Sheet</div><div class='muted'>Preprocessing failed.</div></div></div>"
        return "_Preprocessing failed._", "{}", "\n".join(logs), [], {}, html

    try:
        whisper_result = enhanced_whisper_transcription(cleaned_path, model_name, logs)
    except Exception as e:
        logs.append(f"❌ Whisper transcription failed: {e}")
        html = f"<div class='vs-wrap'><div class='vs-card'><div class='vs-title'>BRU - CQ Sheet</div><div class='muted'>Transcription failed: {escape(str(e))}</div></div></div>"
        return "_Transcription failed._", "{}", "\n".join(logs), [], {}, html

    segments = (whisper_result.get("segments") or [])
    cleaned_segments = detect_and_remove_repetitions(segments, log=logs)

    processed_segments = []
    for seg in cleaned_segments:
        processed_text = post_process_text(seg['text'])
        if processed_text.strip() and len(processed_text.strip()) > 5:
            seg_copy = dict(seg); seg_copy['text'] = processed_text
            processed_segments.append(seg_copy)

    diarization = run_diarization(cleaned_path, hf_token, logs) if enable_diarization else None

    dialogue = build_dialogue(processed_segments, diarization)
    fields   = extract_fields_from_dialogue(dialogue)

    # Ensure required defaults are present
    fields.setdefault("TL Name", DEFAULT_TL_NAME)
    fields.setdefault("QA Name", DEFAULT_QA_NAME)
    fields["Audit Date"] = audit_date_str or fields.get("Audit Date", "—")

    md = format_dialogue_markdown(dialogue)
    payload = {
        "metadata": {
            "total_duration": whisper_result.get('duration', 0),
            "total_speakers": len(set(d['speaker'] for d in dialogue)),
            "total_segments": len(dialogue),
            "model_used": f"whisper-{model_name}",
            "processing_successful": True,
            "anti_repetition_applied": True,
            "diarization_enabled": bool(enable_diarization),
        },
        "dialogue": dialogue,
        "fields": fields,
    }

    # Build the report HTML HERE and return it as an output
    report_html = generate_report_html_from_state(dialogue, fields)
    return md, json.dumps(payload, indent=2, ensure_ascii=False), "\n".join(logs), dialogue, fields, report_html

# -----------------------
# UI
# -----------------------
latest_dialogue_state = gr.State([])
latest_fields_state  = gr.State({})

with gr.Blocks(title="Speech Analysis - QA") as demo:
    gr.Markdown("# 🗣️ Whisper Call Transcriber → 📑 Report\nUpload a call recording, get a clean dialogue and a report like your reference sheet.")

    with gr.Row():
        audio = gr.Audio(type="filepath", label="Upload audio (wav/mp3/m4a...)")
    with gr.Row():
        model_name = gr.Dropdown(choices=["large-v3","large","medium","small","base","tiny"],
                                 value=DEFAULT_MODEL_NAME, label="Whisper model")
        enable_diar = gr.Checkbox(value=True, label="Enable diarization (Pyannote)")
        hf_token = gr.Textbox(value=DEFAULT_HF_TOKEN,
                              label="Hugging Face token (required if diarization enabled)",
                              type="password", placeholder="hf_xxx...")

    run_btn = gr.Button("▶️ Process")

    with gr.Tab("Dialogue"):
        dialogue_md = gr.Markdown()

    with gr.Tab("Report Analysis"):
        gr.Markdown("Report generated automatically from the audio-derived dialogue.")

        # --- NEW: scoring controls (dropdowns) ---
        with gr.Accordion("Set Scores", open=False):
            score_dds = []
            # lay them out 4 per row
            for start in range(0, len(QUALITY_ITEMS), 4):
                with gr.Row():
                    for item in QUALITY_ITEMS[start:start+4]:
                        label = f"{item['sn']}. {item['group']} – {item['text'][:28]}..."
                        dd = gr.Dropdown(
                            choices=SCORE_CHOICES, value="N/A", label=label, interactive=True
                        )
                        score_dds.append(dd)

        # existing HTML (unchanged)
        report_html = gr.HTML(
            value="<div class='vs-wrap'><div class='vs-card'><div class='vs-title'>BRU - CQ Sheet</div><div class='muted'>_No report yet._</div></div></div>"
        )

    # Wire every dropdown to refresh the scorecard HTML when changed
    gr.on(
        [dd.change for dd in score_dds],
        fn=update_scorecard_html,
        inputs=score_dds + [latest_fields_state],
        outputs=report_html,
    )

    with gr.Tab("JSON"):
        json_out = gr.Code(language="json", label="Output JSON")

    with gr.Tab("Logs"):
        logs_out = gr.Textbox(label="Processing Logs", lines=18)
        dialogue_inline = gr.Markdown(value="_Dialogue will appear here_")

    # Single callback returns EVERYTHING, including the report HTML (no .then chain for report)
    run_btn.click(
        transcribe_and_diarize,
        inputs=[audio, model_name, enable_diar, hf_token],
        outputs=[dialogue_md, json_out, logs_out, latest_dialogue_state, latest_fields_state, report_html]
    ).then(
        update_scorecard_html,
        inputs=score_dds + [latest_fields_state],
        outputs=report_html
    ).then(
        lambda md, *_: md, inputs=[dialogue_md], outputs=[dialogue_inline]
    )

if __name__ == "__main__":
    gr.close_all()
    demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2d3f5f6fbcd981b30d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
